In [ ]:
from typing import Any, Dict
import json

####################################
# 1) Imports from the Schnapsen code
####################################
from schnapsen.src.schnapsen.game import (
    GameState, Previous, RegularTrick, ExchangeTrick,
    Card, Rank, Suit, Hand, Talon, Move,
    BotState, Score, RegularMove,
    PlayerPerspective, LeaderPerspective,
    SchnapsenGamePlayEngine
)

##############################################################################
# 2) A function that gathers "initial state + deltas" from a PlayerPerspective
##############################################################################

def perspective_to_dict(persp: PlayerPerspective) -> Dict[str, Any]:
    """
    Convert the perspective's minimal known state into a dictionary.
    We'll store:
      - 'hand': list of card-strings
      - 'known_opponent_cards': list of card-strings
      - 'my_score': (direct_points, pending_points)
      - 'opponent_score': (direct_points, pending_points)
      - 'am_i_leader': bool
      - 'trump_suit': string
      - 'trump_card': string or None
      - 'talon_size': int
      - 'phase': 'ONE' or 'TWO'
    """
    hand_cards = [str(c) for c in persp.get_hand().get_cards()]
    known_opp_cards = [str(c) for c in persp.get_known_cards_of_opponent_hand().get_cards()]

    my_score = persp.get_my_score()
    opp_score = persp.get_opponent_score()

    trump_card = persp.get_trump_card()
    trump_card_str = str(trump_card) if trump_card else None

    return {
        "hand": hand_cards,
        "known_opponent_cards": known_opp_cards,
        "my_score": (my_score.direct_points, my_score.pending_points),
        "opponent_score": (opp_score.direct_points, opp_score.pending_points),
        "am_i_leader": persp.am_i_leader(),
        "trump_suit": str(persp.get_trump_suit()),
        "trump_card": trump_card_str,
        "talon_size": persp.get_talon_size(),
        "phase": persp.get_phase().name
    }

def compute_dict_diff(old_state: Dict[str, Any], new_state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Compare two dictionary representations of a perspective and return
    only the fields that changed, mapped to their new values.
    """
    diff: Dict[str, Any] = {}
    for key, new_val in new_state.items():
        old_val = old_state.get(key, None)
        if new_val != old_val:
            diff[key] = new_val
    return diff

def trick_to_dict(trick) -> Dict[str, Any]:
    """
    Convert a Trick into a minimal dictionary describing the moves played.
    """
    if isinstance(trick, RegularTrick):
        return {
            "type": "RegularTrick",
            "leader_move": str(trick.leader_move),
            "follower_move": str(trick.follower_move),
            "cards_played": [str(c) for c in trick.cards],
        }
    elif isinstance(trick, ExchangeTrick):
        return {
            "type": "ExchangeTrick",
            "exchange": str(trick.exchange),
            "trump_card": str(trick.trump_card),
            "cards_played": [str(c) for c in trick.cards],
        }
    elif trick is not None:
        return {
            "type": "UnknownTrick",
            "cards_played": [str(c) for c in trick.cards],
        }
    else:
        return {}

def gather_deltas_from_history(perspective: PlayerPerspective) -> Dict[str, Any]: # deltas as in "initial state + changes", so that it doesnt save redundant information about previous states. Saves tokens, compute time
    """
    Return a dict with:
        {
            "initial_state": { ... all fields ... },
            "steps": [
               {
                 "index": i,
                 "diff": { ... what changed ... },
                 "trick": { ... if any ... },
                 "valid_moves": [ ... only on last step ... ]
               },
               ...
            ]
        }
    """
    history = perspective.get_game_history()
    if not history:
        return {
            "initial_state": {},
            "steps": []
        }

    # The first snapshot is the earliest perspective + trick
    first_persp, first_trick = history[0]
    prev_state_dict = perspective_to_dict(first_persp)

    data = {
        "initial_state": prev_state_dict,
        "steps": []
    }

    # Iterate from the second item onward
    for i in range(1, len(history)):
        curr_persp, curr_trick = history[i]
        curr_state_dict = perspective_to_dict(curr_persp)
        state_diff = compute_dict_diff(prev_state_dict, curr_state_dict)

        step_info = {
            "index": i,
            "diff": state_diff,
            "trick": trick_to_dict(curr_trick) if curr_trick else None
        }

        # If i == len(history) - 1 => last snapshot => gather valid moves
        if i == len(history) - 1:
            valid_moves_objects = []
            # we call perspective.valid_moves(), which will ask the game engine's validator.
            for mv in curr_persp.valid_moves():
                if mv.is_regular_move():
                    valid_moves_objects.append({
                        "type": "RegularMove",
                        "card": str(mv.as_regular_move().card)
                    })
                elif mv.is_marriage():
                    mar = mv.as_marriage()
                    valid_moves_objects.append({
                        "type": "Marriage",
                        "queen_card": str(mar.queen_card),
                        "king_card": str(mar.king_card)
                    })
                elif mv.is_trump_exchange():
                    te = mv.as_trump_exchange()
                    valid_moves_objects.append({
                        "type": "TrumpExchange",
                        "jack": str(te.jack)
                    })
            step_info["valid_moves"] = valid_moves_objects

        data["steps"].append(step_info)
        prev_state_dict = curr_state_dict

    return data

Creating a fake game scenario to test the code and see what the JSON looks like

In [3]:
#######################################################################################
# 3) Example: Create a partial "3-trick" scenario, then pass the Schnapsen Game engine.
#######################################################################################

game_engine = SchnapsenGamePlayEngine()

# For demonstration, we build a minimal chain of 3 completed tricks, plus a final "current" state.

Qd = Card.get_card(Rank.QUEEN, Suit.DIAMONDS)
As = Card.get_card(Rank.ACE,   Suit.SPADES)
Kd = Card.get_card(Rank.KING,  Suit.DIAMONDS)
Ts = Card.get_card(Rank.TEN,   Suit.SPADES)
Jc = Card.get_card(Rank.JACK,  Suit.CLUBS)
Qh = Card.get_card(Rank.QUEEN, Suit.HEARTS)
Tc = Card.get_card(Rank.TEN,   Suit.CLUBS)

trick1 = RegularTrick(RegularMove(Qd), RegularMove(As))
trick2 = RegularTrick(RegularMove(Kd), RegularMove(Ts))
trick3 = RegularTrick(RegularMove(Jc), RegularMove(Qh))

# We still need intermediate states with some minimal BotStates, so they won't be None.
# We don't pass the engine to those states, we only pass it to the final perspective.
# The engine will be used by get_game_history() to build valid moves, etc.

def dummy_bot_state() -> BotState:
    # Just an example: an empty hand, 0 points
    return BotState(
        implementation=None,
        hand=Hand([], max_size=5),
        score=Score(0, 0),
        won_cards=[]
    )

dummy_leader1 = dummy_bot_state()
dummy_follower1 = dummy_bot_state()
dummy_talon1 = Talon([], trump_suit=Suit.CLUBS)  # minimal talon
state_after_trick1 = GameState(
    leader=dummy_leader1,
    follower=dummy_follower1,
    talon=dummy_talon1,
    previous=None
)
prev1 = Previous(state_after_trick1, trick1, True)

dummy_leader2 = dummy_bot_state()
dummy_follower2 = dummy_bot_state()
dummy_talon2 = Talon([], trump_suit=Suit.CLUBS)
state_after_trick2 = GameState(
    leader=dummy_leader2,
    follower=dummy_follower2,
    talon=dummy_talon2,
    previous=prev1
)
prev2 = Previous(state_after_trick2, trick2, True)

dummy_leader3 = dummy_bot_state()
dummy_follower3 = dummy_bot_state()
dummy_talon3 = Talon([], trump_suit=Suit.CLUBS)
state_after_trick3 = GameState(
    leader=dummy_leader3,
    follower=dummy_follower3,
    talon=dummy_talon3,
    previous=prev2
)
prev3 = Previous(state_after_trick3, trick3, True)

# Now define final "real" BotStates:
bot1_hand = Hand([
    Card.get_card(Rank.ACE, Suit.CLUBS),
    Card.get_card(Rank.KING, Suit.HEARTS),
    Card.get_card(Rank.JACK, Suit.DIAMONDS),
    Card.get_card(Rank.JACK, Suit.SPADES),
    Card.get_card(Rank.TEN, Suit.HEARTS)
], max_size=5)
bot2_hand = Hand([
    Card.get_card(Rank.NINE, Suit.CLUBS),
    Card.get_card(Rank.ACE, Suit.DIAMONDS),
    Card.get_card(Rank.KING, Suit.SPADES),
    Card.get_card(Rank.QUEEN, Suit.SPADES),
    Card.get_card(Rank.TEN, Suit.DIAMONDS)
], max_size=5)

bot1_state = BotState(
    implementation=None,
    hand=bot1_hand,
    score=Score(direct_points=33, pending_points=0),
    won_cards=[]
)
bot2_state = BotState(
    implementation=None,
    hand=bot2_hand,
    score=Score(direct_points=0, pending_points=0),
    won_cards=[]
)

talon_cards = [
    Card.get_card(Rank.NINE, Suit.DIAMONDS),
    Card.get_card(Rank.KING, Suit.CLUBS),
    Tc  # bottom is trump
]
real_talon = Talon(talon_cards)

current_game_state = GameState(
    leader=bot1_state,
    follower=bot2_state,
    talon=real_talon,
    previous=prev3
)

###################################################################################
# 4) Build a LeaderPerspective with the Schnapsen engine, gather deltas, print JSON
###################################################################################

bot1_perspective = LeaderPerspective(current_game_state, engine=game_engine)
info_dict = gather_deltas_from_history(bot1_perspective)

print("===== DELTA-BASED GAME HISTORY JSON =====")
print(json.dumps(info_dict, indent=2))


===== DELTA-BASED GAME HISTORY JSON (with REAL engine) =====
{
  "initial_state": {
    "hand": [],
    "known_opponent_cards": [],
    "my_score": [
      0,
      0
    ],
    "opponent_score": [
      0,
      0
    ],
    "am_i_leader": true,
    "trump_suit": "CLUBS",
    "trump_card": null,
    "talon_size": 0,
    "phase": "TWO"
  },
  "steps": [
    {
      "index": 1,
      "diff": {},
      "trick": {
        "type": "RegularTrick",
        "leader_move": "RegularMove(card=Card.KING_DIAMONDS)",
        "follower_move": "RegularMove(card=Card.TEN_SPADES)",
        "cards_played": [
          "Card.KING_DIAMONDS",
          "Card.TEN_SPADES"
        ]
      }
    },
    {
      "index": 2,
      "diff": {},
      "trick": {
        "type": "RegularTrick",
        "leader_move": "RegularMove(card=Card.JACK_CLUBS)",
        "follower_move": "RegularMove(card=Card.QUEEN_HEARTS)",
        "cards_played": [
          "Card.JACK_CLUBS",
          "Card.QUEEN_HEARTS"
        ]
      }

Creating JSON

In [4]:
game_state_json_str = json.dumps(info_dict, indent=2)

API call (example for openai, just placeholder fro now)

In [ ]:
import openai

openai.api_key = "YOUR_OPENAI_API_KEY"

def call_llm_for_move(game_state_json: dict) -> dict:
    # 1) Convert game state to string
    game_state_str = json.dumps(game_state_json, indent=2)

    # 2) Build a prompt with instructions
    prompt = f"""
You are playing a game of Schnapsen. Here is a JSON structure describing the entire state:
{game_state_json_str}

Please pick exactly one move from the "valid_moves" of the final step.
Return ONLY valid JSON with the fields:
- "type": one of ["RegularMove", "Marriage", "TrumpExchange"]
- If "RegularMove", also "card": ...
- If "Marriage", also "queen_card" and "king_card"
- If "TrumpExchange", also "jack"
No extra text or explanation—just the JSON.

Example:
{{
  "type": "RegularMove",
  "card": "Card(rank=ACE, suit=HEARTS)"
}}
"""

    # 3) Call the LLM 
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # or another model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0  # for deterministic output
    )

    # 4) LLM's reply
    llm_text = response["choices"][0]["message"]["content"]

    # 5) Parse the text as JSON (the LLM should be returning something like {"type": "RegularMove", ...})
    try:
        move_dict = json.loads(llm_text)
    except json.JSONDecodeError:
        # for invalid JSON or extra text
        raise ValueError(f"LLM returned invalid JSON:\n{llm_text}")

    return move_dict


Still need to write helper function to convert LLM chosen move into Move object for playing.
Also, probably need better prompt